In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from image_processing_utilities.functions import validation_dataset_generator as dataset_generator
from denoising_functions import total_variation_samples
from image_processing_utilities.functions import metrics_samples
from image_processing_utilities.functions import validation_dataset_generator
from image_processing_utilities.functions import ssim_batch
from image_processing_utilities.functions import get_samples
import numpy as np
import json

In [ ]:
dataset = 'SIDD'
method = 'Gaussian'
color = True

x_noisy, x_gt = dataset_generator()

In [ ]:
samples = json.load(open('config.json'))[dataset]
x_noisy_samples = get_samples(x_noisy, samples)
x_gt_samples = get_samples(x_gt, samples)

# Sample Images
image_samples = [1, 10, 17, 23]
image_crops = [2, 4, 7, 11]
samples_noisy = np.array([[x_noisy[i, j] for j in image_crops] for i in image_samples])
samples_gt = np.array([[x_gt[i, j] for j in image_crops] for i in image_samples])

In [ ]:
# Create subplot figure with 2 rows and 2 columns
fig = make_subplots(rows=4, cols=4, subplot_titles=(
    "Noisy 01", "GT 01", "Noisy 02", "GT 02", 
    "Noisy 03", "GT 03", "Noisy 04", "GT 04",
    "Noisy 05", "GT 05", "Noisy 06", "GT 06", 
    "Noisy 07", "GT 07", "Noisy 08", "GT 08"
))

# Add images to the subplots
fig.add_trace(go.Image(z=samples_noisy[0][1]), row=1, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[0][1]), row=1, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][0]), row=1, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[1][0]), row=1, col=4)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][3]), row=2, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[1][3]), row=2, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[2][0]), row=2, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[2][0]), row=2, col=4)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][0]), row=3, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][0]), row=3, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][1]), row=3, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][1]), row=3, col=4)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][2]), row=4, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][2]), row=4, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][3]), row=4, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][3]), row=4, col=4)  # Ground Truth

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=1000, 
                  width=1200, 
                  title_text="Noisy and Ground Truth Images",
                  margin=dict(l=20, r=20, t=80, b=20),  # Reduce margins
)
fig.show()

In [ ]:
# Generate initial metrics:
print('\nMetrics:')
initial_metrics = metrics_samples(x_noisy.astype(np.single), x_gt.astype(np.single), color=color)
print('MAE: ', initial_metrics['MAE'])
print('MSE: ', initial_metrics['MSE'])
print('SSIM: ', initial_metrics['SSIM'])
print('PSNR: ', initial_metrics['PSNR'])


In [ ]:
# Grid search find the best parameters:
lambdas = list(np.arange(0.05, 2.55, .05))


best_loss = 1e4
best_lambda = None
best_index = None

metrics = np.zeros((len(lambdas)))
for i, lambda_val in enumerate(lambdas):
    test = total_variation_samples(x_noisy_samples, lambda_val, color)
    avg_loss = 1 - ssim_batch(test.astype(np.single), x_gt_samples.astype(np.single))  # SSIM Loss
    metrics[i] = avg_loss

    if avg_loss < best_loss:
        best_index = i
        best_lambda = lambda_val
        best_loss = avg_loss

    print('Lambda: {lambda_val}, Loss: {loss}'.format(lambda_val=lambda_val, loss=avg_loss))
    del test


In [ ]:
print('Best Parameters -- Lambda: {lambda_val}'.format(lambda_val=best_lambda))
denoised = total_variation_samples(x_noisy, best_lambda, color)


In [ ]:
# Generate final metrics:
final_metrics = metrics_samples(denoised.astype(np.single), x_gt.astype(np.single), color=color)

print('\nDenoised VALUES')
print('MAE: ', final_metrics['MAE'])
print('MSE: ', final_metrics['MSE'])
print('SSIM: ', final_metrics['SSIM'])
print('PSNR: ', final_metrics['PSNR'])


In [ ]:
denoised_samples = total_variation_samples(samples_noisy, best_lambda, color)

In [ ]:
# Create subplot figure with 2 rows and 2 columns
fig = make_subplots(rows=4, cols=6, subplot_titles=(
    "Noisy 01", "Denoised 01", "GT 01", "Noisy 02", "Denoised 02", "GT 02", 
    "Noisy 03", "Denoised 03", "GT 03", "Noisy 04", "Denoised 04", "GT 04",
    "Noisy 05", "Denoised 05", "GT 05", "Noisy 06", "Denoised 06", "GT 06", 
    "Noisy 07", "Denoised 07", "GT 07", "Noisy 08", "Denoised 08", "GT 08"
))

# Add images to the subplots
fig.add_trace(go.Image(z=samples_noisy[0][1]), row=1, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[0][1]), row=1, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[0][1]), row=1, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][0]), row=1, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[1][0]), row=1, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][0]), row=1, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][3]), row=2, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[1][3]), row=2, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][3]), row=2, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[2][0]), row=2, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[2][0]), row=2, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[2][0]), row=2, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][0]), row=3, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[3][0]), row=3, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][0]), row=3, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][1]), row=3, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[3][1]), row=3, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][1]), row=3, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][2]), row=4, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[3][2]), row=4, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][2]), row=4, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][3]), row=4, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised_samples[3][3]), row=4, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][3]), row=4, col=6)  # Ground Truth

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=1000, 
                  width=1200, 
                  # title_text="Noisy and Ground Truth Images",
                  margin=dict(l=40, r=40, t=40, b=20),  # Reduce margins
)
fig.show()

In [ ]:
# Create the plot
fig = go.Figure(data=go.Scatter(x=lambdas, y=metrics, mode='lines'))
fig.update_layout(height=400, width=1000,  xaxis_title='Lambda Value', yaxis_title='Loss')
fig.show()